### Modules

In [ ]:
import tensorflow_addons as tfa
from tensorflow.keras import applications
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import datetime
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model, Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Dropout, Flatten, Dense,Input,GlobalAveragePooling2D,AveragePooling2D
import datetime
import random
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

### Hand-Craft ResNet-18

In [ ]:
from Network.ResNet import ResNet
import tensorflow_addons as tfa
model = ResNet([2,2,2,2])
model.build(input_shape=(None,H,W,C))

### ResNet-50 from Keras

In [ ]:
# Transfer Learning - Tryouts

base_model = applications.resnet50.ResNet50(weights='imagenet', include_top=False,layers=tf.keras.layers)  
#base_model = applications.xception.Xception(weights='imagenet', include_top=False, input_shape=[resize_H,resize_W,channel]) 


for layer in base_model.layers[:140]:  # Keep the pretrained params
    layer.trainable = False
for layer in base_model.layers[140:]:  # Keep the pretrained params
    layer.trainable = True
    
x = base_model.output  # 
x = GlobalAveragePooling2D()(x)  
x = Dense(1024, activation='relu', name='fc1',kernel_regularizer=keras.regularizers.l2(0.0001))(x)  
x = Dropout(0.5)(x)  # Droupout 0.6
x = Dense(512, activation='relu', name='fc2',kernel_regularizer=keras.regularizers.l2(0.0001))(x)
#x = Dropout(0.5)(x)
predictions = Dense(5, activation='softmax', name='predictions')(x)  

model = Model(base_model.input,predictions)  

### Hyperparameters

In [ ]:
#optimizer = tfa.optimizers.AdamW(lr=3e-4, weight_decay=3e-4)
optimizer = tf.optimizers.Adam(lr=3e-4)
loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer,loss=loss,metrics=['accuracy'])
model.summary()

### Tranining Strategies and Implementation

In [ ]:
reduce = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.6, patience=4, mode='auto')
early = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=5, mode='auto')
model.fit(X_train,y_train, epochs=30, shuffle = True, validation_split=0.1, batch_size = 8, callbacks=[reduce, early])